In [ ]:
import pandas as pd
import re

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def get_last_three_months(current_month, current_year):
    """Return tuples of the last three months with their years."""
    months_years = []
    for i in range(3):
        month = (current_month - i) % 12 or 12
        year = current_year if month <= current_month else current_year - 1
        months_years.append((month, year))
    return list(reversed(months_years))

def append_data_to_df(paste_df, sheet_name, period_value, row_value, entity, function, mica, cost_type):
    """Append data to the DataFrame based on cost type (DC or HC)."""
    paste_row = {
        "File Name": sheet_name,
        "Level 3": '',
        "Level 4": '',
        "Cost Grouping": '',
        "Cost Type": '',
        "Finance Region": '',
        "Attribute Type": 'MoM',  # We'll modify this later for other attribute types
        "Period": period_value,
        "Cost": row_value if cost_type == 'Cost' else '',
        "FTE/Contractor": row_value if cost_type == 'FTE/Contractor' else '',
        "Country": '',
        "Level 3.5": '',
        "Level 4.5": '',
        "Entity": entity,
        "Function": function,
        "Mapped Country": '',
        "MICA": mica
    }
    paste_df = paste_df.append(paste_row, ignore_index=True)
    return paste_df

def get_mom_data(sheet, current_month, current_year):
    """Extract the MoM data from a given sheet."""
    last_three_months = get_last_three_months(current_month, current_year)
    mom_columns = []
    for col in sheet.columns:
        match = re.match(r'M(\d{2}) (\d{4})', col)  # Match Mxx YYYY pattern
        if match:
            month_num = int(match.group(1))  # Extract month number
            year = int(match.group(2))  # Extract year
            if (month_num, year) in last_three_months:
                mom_columns.append(col)
    return mom_columns

def process_mom_data(processed_file_path, current_month, current_year):
    """Process the MoM data for sheets with 'Actual' in the name."""
    xls = pd.ExcelFile(processed_file_path)
    paste_df = pd.DataFrame(columns=[
        "File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
        "Finance Region", "Attribute Type", "Period", "Cost", 
        "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
        "Function", "Mapped Country", "MICA"
    ])

    for sheet_name in xls.sheet_names:
        if 'Actual' in sheet_name.lower():  # Process only 'Actual' sheets
            sheet = pd.read_excel(xls, sheet_name=sheet_name)
            mom_columns = get_mom_data(sheet, current_month, current_year)

            for index, row in sheet.iterrows():
                entity = row.get('Entity', '')
                function = row.get('RTN Level 4', '')
                mica = row.get('Account', '')

                for col in mom_columns:
                    month_num, year = map(int, re.findall(r'\d+', col))
                    period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"
                    row_value = row[col]

                    if 'DC' in sheet_name:
                        paste_df = append_data_to_df(paste_df, sheet_name, period_value, row_value, entity, function, mica, 'Cost')
                    elif 'HC' in sheet_name:
                        paste_df = append_data_to_df(paste_df, sheet_name, period_value, row_value, entity, function, mica, 'FTE/Contractor')

    return paste_df

# Example usage
current_month = 4  # April
current_year = 2024
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
paste_df = process_mom_data(processed_file_path, current_month, current_year)

# Write the DataFrame to a new sheet
paste_df.to_excel('Paste_MoM.xlsx', index=False)


In [ ]:
import pandas as pd
import re

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create an empty DataFrame to collect all data for the "Paste" sheet
    paste_df = pd.DataFrame(columns=["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
                                     "Finance Region", "Attribute Type", "Period", "Cost", 
                                     "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
                                     "Function", "Mapped Country", "MICA"])

    # Process each sheet
    for sheet_name in xls.sheet_names:
        # Only process sheets containing 'Actual', 'Target', or 'Fcst'
        if re.search(r'Actual|Target|Fcst', sheet_name, re.IGNORECASE):
            # Only load required columns from the sheet
            sheet = pd.read_excel(xls, sheet_name=sheet_name, usecols=lambda x: re.match(r'Entity|RTN Level 4|Account|M\d{2}|FY|YTD', x))

            # Ensure relevant columns exist
            if not {'Entity', 'RTN Level 4', 'Account'}.issubset(sheet.columns):
                continue

            # Extract relevant columns
            entity_col = sheet['Entity']
            function_col = sheet['RTN Level 4']
            account_col = sheet['Account']

            # Identify cost type (DC for Cost, HC for FTE/Contractor)
            cost_type = 'Cost' if 'DC' in sheet_name else 'FTE/Contractor'

            # 1. MoM (Last 3 Months Data)
            month_cols = [col for col in sheet.columns if re.match(r'M\d{2}', col)]
            for month_col in month_cols:
                month_num = int(month_col[1:3])
                period_value = f"{month_number_to_name(month_num)}-{str(current_year)[-2:]}"

                if (current_month - month_num) % 12 in [0, 1, 2]:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'MoM',
                        'Period': period_value,
                        cost_type: sheet[month_col]
                    })], ignore_index=True)

            # 2. YTD (Summing up values up to current month)
            ytd_cols = [col for col in sheet.columns if re.match(r'M\d{2}', col) and int(col[1:3]) <= current_month]
            if ytd_cols:
                sheet['YTD Total'] = sheet[ytd_cols].sum(axis=1)

                ytd_period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                if 'Target' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Target',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

                elif 'Fcst' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Forecast',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

                elif 'Actual' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Actual',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

            # 3. Prior Year FY
            prior_fy_cols = [col for col in sheet.columns if f'FY {current_year - 1}' in col]
            for prior_fy_col in prior_fy_cols:
                prior_fy_period_value = f"Dec-{str(current_year - 1)[-2:]}"
                paste_df = pd.concat([paste_df, pd.DataFrame({
                    'File Name': sheet_name,
                    'Entity': entity_col,
                    'Function': function_col,
                    'Account': account_col,
                    'Attribute Type': 'Prior Year FY',
                    'Period': prior_fy_period_value,
                    cost_type: sheet[prior_fy_col]
                })], ignore_index=True)

            # 4. Prior Year YTD
            prior_ytd_cols = [col for col in sheet.columns if f'YTD {current_year - 1}' in col]
            for prior_ytd_col in prior_ytd_cols:
                prior_ytd_period_value = f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}"
                paste_df = pd.concat([paste_df, pd.DataFrame({
                    'File Name': sheet_name,
                    'Entity': entity_col,
                    'Function': function_col,
                    'Account': account_col,
                    'Attribute Type': 'Prior Year YTD',
                    'Period': prior_ytd_period_value,
                    cost_type: sheet[prior_ytd_col]
                })], ignore_index=True)

    # Write the final DataFrame to an Excel file
    paste_df.to_excel('final_output_file.xlsx', sheet_name='Paste', index=False)

# Example usage
current_month = 6  # Example: June
current_year = 2024  # Example: Current year
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
